In [ ]:
# Create folders for the project
import os

base_path = "HogwartsQ&A"
os.makedirs(base_path, exist_ok=True)
os.makedirs(os.path.join(base_path, 'data'), exist_ok=True)
os.makedirs(os.path.join(base_path, 'models'), exist_ok=True)
os.makedirs(os.path.join(base_path, 'api'), exist_ok=True)
os.makedirs(os.path.join(base_path, 'web'), exist_ok=True)
os.makedirs(os.path.join(base_path, 'static'), exist_ok=True)

print(f"Project folder structure created at: {base_path}")

In [5]:
# Download the book from a URL (if accessible)
import requests

url = 'https://ia902903.us.archive.org/12/items/FantasyFictionebookcollection/Harry%20Potter/3%20-%20Harry%20Potter%20and%20the%20Prisoner%20of%20Azkaban.pdf'
response = requests.get(url)
with open(os.path.join(base_path, 'data', 'Harry_Potter_and_Prisoner_of_Azkaban.pdf'), 'wb') as f:
    f.write(response.content)

In [7]:
!pip install pdfminer.six sentence-transformers faiss-cpu fastapi uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 33.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 34.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.8/381.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.7 MB/s eta 0:00:00a 0:00:01


In [9]:
from pdfminer.high_level import extract_text

# Define file paths
pdf_path = f"{base_path}/data/Harry_Potter_and_Prisoner_of_Azkaban.pdf"
text_output_path = f"{base_path}/data/Harry_Potter_Parsed.txt"

# Extract text from the PDF
text = extract_text(pdf_path)

# Save the parsed text to a file
with open(text_output_path, 'w', encoding='utf-8') as f:
    f.write(text)

print("PDF parsed and saved as text successfully!")

PDF parsed and saved as text successfully!


In [12]:
def split_text_into_chunks(text, chunk_size=150):
    """Splits the text into smaller chunks based on word count."""
    words = text.split()
    chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

# Read the parsed text file
with open(text_output_path, 'r', encoding='utf-8') as f:
    book_text = f.read()

# Split the text into chunks of 150 words each
text_chunks = split_text_into_chunks(book_text)

# Show a few sample chunks
for i, chunk in enumerate(text_chunks[:3]):
    print(f"Chunk {i+1}: {chunk[:200]}...\n")

Chunk 1: Harry Potter and the Prisoner of Azkaban by J. K. Rowling Illustrations By Mary Grandpré Arthur A. Levine Books An Imprint of Scholastic Press. To Jill Prewett and Aine Kiely, the GodMothers of Swing ...

Chunk 2: Rowling, J. K. Harry Potter and the Prisoner of Azkaban / by J. K. Rowling. p. cm. Sequel to: Harry Potter and the Chamber of Secrets Summary: During his third year at Hogwarts School for Witchcraft a...

Chunk 3: midnight, and he was lying on his stomach in bed, the blankets drawn right over his head like a tent, a ﬂashlight in one hand and a large leather-bound book (A History of Magic by Bathilda Bagshot) pr...



In [16]:
!pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.2 MB/s eta 0:00:00 0:00:01


In [18]:
from sentence_transformers import SentenceTransformer
import pickle

# Load the pre-trained model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings for each chunk
embeddings = model.encode(text_chunks)

# Save the embeddings and chunks for future use
with open(f"{base_path}/models/book_embeddings.pkl", 'wb') as f:
    pickle.dump((text_chunks, embeddings), f)

print(f"Generated and saved embeddings for {len(embeddings)} text chunks.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generated and saved embeddings for 733 text chunks.


In [24]:
import os

file_path = '/Users/harshavardhangowda/HogwartsQ&A/models/book_embeddings.pkl'
if os.path.exists(file_path):
    print(f"File found at: {file_path}")
else:
    print("File not found. Please check the path or regenerate the embeddings.")

File found at: /Users/harshavardhangowda/HogwartsQ&A/models/book_embeddings.pkl


In [26]:
import faiss
import numpy as np

# Convert embeddings to numpy array
embeddings_np = np.array(embeddings, dtype='float32')

# Create a FAISS index
index = faiss.IndexFlatL2(embeddings_np.shape[1])  # L2 distance measure
index.add(embeddings_np)  # Add the embeddings to the index

print(f"Indexed {len(embeddings)} text chunks in FAISS.")

Indexed 733 text chunks in FAISS.


In [28]:
def search_chunks(query, index, model, chunks, top_n=5):
    """Searches for the most relevant chunks for a given query."""
    query_embedding = model.encode([query])[0]  # Encode the query
    distances, indices = index.search(np.array([query_embedding], dtype='float32'), top_n)
    
    # Retrieve top N relevant chunks
    results = [{"text": chunks[idx], "score": distances[0][i]} for i, idx in enumerate(indices[0])]
    return results

# Test the search with a sample query
query = "Who created the Marauder's Map?"
results = search_chunks(query, index, model, text_chunks)

# Display the results
for result in results:
    print(f"Text: {result['text'][:200]} | Score: {result['score']}")

Text: began to spread like a spider’s web from the point that George’s wand had touched. They joined each other, they crisscrossed, they fanned into every corner of the parchment; then words began to blosso | Score: 0.8207749128341675
Text: up, Remus,” snarled Black, who was still watching Scabbers with a horrible sort of hunger on his face. “I’m getting there, Sirius, I’m getting there … well, highly exciting possibilities were open to  | Score: 1.021514654159546
Text: said Lupin shortly. He glanced around the empty entrance hall and lowered his voice. “I happen to know that this map was conﬁscated by Mr. Filch many years ago. Yes, I know it’s a map,” he said as Har | Score: 1.0358514785766602
Text: right … that’s why we called him Prongs.” Lupin threw his last few books into his case, closed the desk drawers, and turned to look at Harry. “Here — I brought this from the Shrieking Shack last night | Score: 1.0827686786651611
Text: the heading of the map. “We owe them so much.” “Noble m

In [30]:
def ask_question(question):
    """Simulates the API response by using the search function."""
    results = search_chunks(question, index, model, text_chunks)
    return {"question": question, "answer": results[0]['text'] if results else "No relevant answer found."}

# Test with a question
response = ask_question("What is the significance of the Marauder's Map?")
print(response)

{'question': "What is the significance of the Marauder's Map?", 'answer': 'began to spread like a spider’s web from the point that George’s wand had touched. They joined each other, they crisscrossed, they fanned into every corner of the parchment; then words began to blossom across the top, great, curly green words, that proclaimed: Messrs. Moony, Wormtail, Padfoot, and Prongs Purveyors of Aids to Magical Mischief-Makers are proud to present THE MARAUDER’S MAP in the top It was a map showing every detail of the Hogwarts castle and grounds. But the truly remarkable thing were the tiny ink dots moving around it, each labeled with a name in minuscule writing. Astounded, Harry bent over it. A labeled left corner showed that Professor dot Dumbledore was pacing his study; the caretaker’s cat, Mrs. Norris, was prowling the second ﬂoor; and Peeves the Poltergeist was currently bouncing around the trophy room. And as Harry’s eyes traveled up and down the familiar corridors, he'}


In [32]:
# Import necessary libraries
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

def query_pipeline(query, model, index, chunks, top_n=3):
    """Handles the end-to-end query pipeline for retrieving the most relevant chunks."""
    # Step 1: Convert the query to an embedding
    query_embedding = model.encode([query])[0]
    
    # Step 2: Perform similarity search using FAISS index
    distances, indices = index.search(np.array([query_embedding], dtype='float32'), top_n)

    # Step 3: Retrieve top N relevant chunks
    top_chunks = [{"text": chunks[idx], "score": distances[0][i]} for i, idx in enumerate(indices[0])]
    
    # Step 4: Format the final response using the most relevant chunk
    formatted_response = f"Answer: {top_chunks[0]['text']}\n\nAdditional Information:\n"
    for i, chunk in enumerate(top_chunks[1:], start=2):
        formatted_response += f"Chunk {i}: {chunk['text'][:200]}... | Score: {chunk['score']}\n"
    
    return formatted_response

# Test the query pipeline with a sample question
sample_question = "Who created the Marauder's Map?"
response = query_pipeline(sample_question, model, index, text_chunks)
print(response)

Answer: began to spread like a spider’s web from the point that George’s wand had touched. They joined each other, they crisscrossed, they fanned into every corner of the parchment; then words began to blossom across the top, great, curly green words, that proclaimed: Messrs. Moony, Wormtail, Padfoot, and Prongs Purveyors of Aids to Magical Mischief-Makers are proud to present THE MARAUDER’S MAP in the top It was a map showing every detail of the Hogwarts castle and grounds. But the truly remarkable thing were the tiny ink dots moving around it, each labeled with a name in minuscule writing. Astounded, Harry bent over it. A labeled left corner showed that Professor dot Dumbledore was pacing his study; the caretaker’s cat, Mrs. Norris, was prowling the second ﬂoor; and Peeves the Poltergeist was currently bouncing around the trophy room. And as Harry’s eyes traveled up and down the familiar corridors, he

Additional Information:
Chunk 2: up, Remus,” snarled Black, who was still watching